In [1]:
import os
import torch
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm


# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset path
data_dir = r"C:\\Users\\sahba\\Downloads\\archive (2)\\IMG_CLASSES"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", full_dataset.classes)

Using device: cuda
Classes: ['C', 'M', 'MN', 'R', 'SK', 'WM']


In [2]:


# Class weights (update based on your dataset's class distribution)
class_counts = [
    2697,  # Melanoma
    2215,  # Carcinoma
    6946,  # Melanocytic_Nevi
    1827,  # Seborrheic_Keratoses
    1644,  # TineaRingwormCandidiasis
    1888   # WartsMolluscum
]
class_counts_tensor = torch.tensor(class_counts, dtype=torch.float)
class_weights = 1.0 / class_counts_tensor
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = class_weights.to(device)


In [3]:

# Constants
num_epochs = 20
num_folds = 5
batch_size = 32
num_classes = 6
learning_rate = 0.001

# K-Fold setup
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
dataset = full_dataset
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n========== Fold {fold + 1} ==========")

    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Load AlexNet
    model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
    for p in model.parameters():
        p.requires_grad = False
    for idx in (3, 4):
        for p in model.features[idx].parameters():
            p.requires_grad = True

    in_feat = model.classifier[6].in_features
    model.classifier[6] = nn.Sequential(
        nn.Linear(in_feat, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )
    model = model.to(device)

    optimizer = optim.Adam([
        {'params': model.features[3].parameters(), 'lr': 1e-6},
        {'params': model.features[4].parameters(), 'lr': 1e-5},
        {'params': model.classifier[6].parameters(), 'lr': 1e-3}
    ])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)
    criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in tqdm(train_loader, leave=False, desc=f"Fold {fold+1} - Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        scheduler.step(running_loss / len(train_loader))

    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100. * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    fold_accuracies.append(acc)
    print(f"Fold {fold+1} Accuracy: {acc:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

print("\n========== Cross-Validation Summary ==========")
for i, acc in enumerate(fold_accuracies):
    print(f"Fold {i+1}: {acc:.2f}%")
print(f"Average Accuracy: {np.mean(fold_accuracies):.2f}%")


========== Fold 1 ==========


Fold 1 Accuracy: 84.38%
              precision    recall  f1-score   support

           C       0.87      0.95      0.91       433
           M       0.80      0.87      0.83       540
          MN       0.95      0.88      0.91      1409
           R       0.71      0.82      0.76       325
          SK       0.77      0.73      0.75       357
          WM       0.72      0.70      0.71       375

    accuracy                           0.84      3439
   macro avg       0.80      0.82      0.81      3439
weighted avg       0.85      0.84      0.84      3439


========== Fold 2 ==========


Fold 2 Accuracy: 84.27%
              precision    recall  f1-score   support

           C       0.91      0.91      0.91       447
           M       0.75      0.90      0.82       544
          MN       0.95      0.87      0.91      1382
           R       0.75      0.74      0.75       319
          SK       0.74      0.80      0.77       380
          WM       0.75      0.69      0.71       367

    accuracy                           0.84      3439
   macro avg       0.81      0.82      0.81      3439
weighted avg       0.85      0.84      0.84      3439


========== Fold 3 ==========


Fold 3 Accuracy: 82.76%
              precision    recall  f1-score   support

           C       0.84      0.95      0.89       447
           M       0.78      0.84      0.81       557
          MN       0.93      0.87      0.90      1361
           R       0.75      0.70      0.72       319
          SK       0.68      0.78      0.73       355
          WM       0.76      0.67      0.71       400

    accuracy                           0.83      3439
   macro avg       0.79      0.80      0.79      3439
weighted avg       0.83      0.83      0.83      3439


========== Fold 4 ==========


Fold 4 Accuracy: 82.47%
              precision    recall  f1-score   support

           C       0.85      0.95      0.90       455
           M       0.73      0.84      0.78       552
          MN       0.93      0.85      0.89      1356
           R       0.74      0.75      0.75       350
          SK       0.77      0.72      0.74       372
          WM       0.73      0.73      0.73       354

    accuracy                           0.82      3439
   macro avg       0.79      0.81      0.80      3439
weighted avg       0.83      0.82      0.83      3439


========== Fold 5 ==========


Fold 5 Accuracy: 82.76%
              precision    recall  f1-score   support

           C       0.88      0.90      0.89       433
           M       0.69      0.90      0.78       504
          MN       0.96      0.85      0.90      1417
           R       0.71      0.82      0.76       331
          SK       0.71      0.77      0.74       363
          WM       0.83      0.64      0.72       391

    accuracy                           0.83      3439
   macro avg       0.80      0.81      0.80      3439
weighted avg       0.84      0.83      0.83      3439


========== Cross-Validation Summary ==========
Fold 1: 84.38%
Fold 2: 84.27%
Fold 3: 82.76%
Fold 4: 82.47%
Fold 5: 82.76%
Average Accuracy: 83.33%
